Import of necessary packages/libraries

In [1]:
import numpy as np
import pandas as pd 
import sklearn as sl
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import yfinance as yf

Version check

In [2]:
numpy_version = np.__version__
pandas_version = pd.__version__
sklearn_version = sl.__version__
matplotlib_version = matplotlib.__version__
seaborn_version = sns.__version__
yahoo_version = yf.__version__
print(f"Numpy version: {numpy_version}")
print(f"Pandas version: {pandas_version}")
print(f"Sklearn version: {sklearn_version}")
print(f"Matplotlib version: {matplotlib_version}")
print(f"Seaborn version: {seaborn_version}")
print(f"yFinance version: {yahoo_version}")

Numpy version: 2.3.5
Pandas version: 2.3.3
Sklearn version: 1.7.1
Matplotlib version: 3.10.6
Seaborn version: 0.13.2
yFinance version: 0.2.66


Import of data

In [3]:
df = pd.read_excel(r"Top5_Indices.xlsx", sheet_name = "Continuous")
df['Date'] = pd.to_datetime(df['Date'])
print(df.head())

        Date      S&P 500     Dow Jones  Nasdaq Composite  Russell 2000  \
0 2000-01-03  1455.219971  11357.509766       4131.149902    496.420013   
1 2000-01-04  1399.420044  10997.929688       3901.689941    478.380005   
2 2000-01-05  1402.109985  11122.650391       3877.540039    478.829987   
3 2000-01-06  1403.449951  11253.259766       3727.129883    475.339996   
4 2000-01-07  1441.469971  11522.559570       3882.620117    488.309998   

    Nasdaq 100  DayNumber  
0  3790.550049          1  
1  3546.199951          2  
2  3507.310059          3  
3  3340.810059          4  
4  3529.600098          5  


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9469 entries, 0 to 9468
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              9469 non-null   datetime64[ns]
 1   S&P 500           9469 non-null   float64       
 2   Dow Jones         9469 non-null   float64       
 3   Nasdaq Composite  9469 non-null   float64       
 4   Russell 2000      9469 non-null   float64       
 5   Nasdaq 100        9469 non-null   float64       
 6   DayNumber         9469 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 518.0 KB
None


In [5]:
print(df.shape)

(9469, 7)


Linear Regression

In [7]:
from sklearn.linear_model import LinearRegression

x = df['DayNumber'].values.reshape(-1, 1)  # Reshape for sklearn
y1 = df['S&P 500'].values
y2 = df['Dow Jones'].values
y3 = df['Nasdaq Composite'].values
y4 = df['Russell 2000'].values
y5 = df['Nasdaq 100'].values

model1 = LinearRegression()
model1.fit(x, y1)
model2 = LinearRegression()
model2.fit(x, y2)
model3 = LinearRegression()
model3.fit(x, y3)
model4 = LinearRegression()
model4.fit(x, y4)
model5 = LinearRegression()
model5.fit(x, y5)

print("S&P 500 R^2 Score:", model1.score(x, y1))
print("Dow Jones Score:", model2.score(x, y2))
print("Nasdaq Composite Score:", model3.score(x, y3))
print("Russell 2000 Score:", model4.score(x, y4))
print("Nasdaq 100 Score:", model5.score(x, y5))

S&P 500 R^2 Score: 0.748480956063795
Dow Jones Score: 0.8160733496767216
Nasdaq Composite Score: 0.7218461994839633
Russell 2000 Score: 0.8735624248814405
Nasdaq 100 Score: 0.6986548035800297
